In [1]:
import pandas as pd
import json
import csv

In [2]:
t1 = ["2018/" + str(i) for i in range(201800000,201802792)]
t2 = ["2019/" + str(i) for i in range(201900000,201903082)]
t3 = ["2020/" + str(i) for i in range(202000000,202003393)]
t4 = ["2021/" + str(i) for i in range(202100000,202103815)]
t5 = ["2022/" + str(i) for i in range(202200000,202204244)]
t6 = ["2023/" + str(i) for i in range(202300000,202302890)]

t = t1 + t2 + t3 + t4 + t5 + t6

In [3]:
len(t)

20216

In [43]:
i = 0
for dir in t:
    f = open('../data/' + dir)
    data = json.load(f)
    f.close()

    #main csv file
    title = data['abstracts-retrieval-response']['item']['bibrecord']['head']['citation-title']
    abstract = data['abstracts-retrieval-response']['item']['bibrecord']['head']['abstracts']
    public_date = data['abstracts-retrieval-response']['item']['ait:process-info']['ait:date-delivered']['@timestamp']
    authors = []
    for group in data['abstracts-retrieval-response']['item']['bibrecord']['head']['author-group']['author']:
        if (type(group) == list):
            authors = [x for x in group['ce:indexed-name']]
        else:
            authors.append(group['ce:indexed-name'])
    author_keywords = [keyword['$'] for keyword in data['abstracts-retrieval-response']['item'].get('authkeywords', {}).get('author-keyword', [])]
    reference_authors = []
    for reference in data['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference']:
        if 'ref-authors' in reference.get('ref-info', {}):
            for author in reference['ref-info']['ref-authors'].get('author', []):
                reference_authors.append(author.get('ce:indexed-name', ''))

    abbrev = set()
    for subject in data['abstracts-retrieval-response']['subject-areas']['subject-area']:
        abbrev.add(subject['@abbrev'])

    abbrev = list(abbrev)

    with open('main_file.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([title, abstract, public_date, authors, author_keywords, reference_authors, abbrev])

    #author to allications file
    author_affiliation_data = []
    for group in data['abstracts-retrieval-response']['item']['bibrecord']['head']['author-group']:
        for aut in group['author']:
            author_affiliation_data.append([aut['preferred-name']['ce:indexed-name'], group['affiliation']['@afid']])

    with open('author2afflication.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(author_affiliation_data)

    #afflication file
    affiliation_data = []
    for group in data['abstracts-retrieval-response']['item']['bibrecord']['head']['author-group']:
        #for affiliation in group['affiliation']:
        country = group['affiliation']['country']
        city = group['affiliation']['city']
        organizations = []
        if (type(group['affiliation']['organization']) == list):
            for org in group['affiliation']['organization']:
                organizations.append(org['$'])
        else:
            organizations.append(org['$'])
        afid = group['affiliation']['@afid']
        affiliation_data.append([afid, country, city, organizations])

    with open('afflication.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(affiliation_data)

    #reference file
    reference_data = []
    for reference in data['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference']:
        ref_info = reference['ref-info']
        if 'ref-website' in ref_info:
            website_data = ref_info['ref-website']
            if isinstance(website_data, list):
                for website in website_data:
                    attributes = list(website.keys())
                    link = website['$']
                    reference_data.append(attributes + [link])
            else:
                attributes = list(website_data.keys())
                link = website_data['ce:e-address']['$']
                reference_data.append(attributes + [link])

    with open('reference.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(reference_data)

    i += 1
    if (i % 256 == 0):
        print(i + " files read successfully")

TypeError: list indices must be integers or slices, not str